# Title : Crawler_Shopee_EC 

Date : 2020.07.02

Update : 2020.08.06

Author : Mao-Zhou Wang

Email : maozhou0112@gmail.com

In [ ]:
import requests
import json
import time
from fake_useragent import UserAgent
import pandas as pd
from bs4 import BeautifulSoup as bs
import math
import pyodbc
import numpy as np

In [2]:
#server = '***,***' #伺服器名稱,port
#database = '***'
#username = '***'
#password = '***'
cnxn = pyodbc.connect('DRIVER={sql server};SERVER=' + server + ';DATABASE=' +
                      database + ';UID=' + username + ';PWD=' + password)
cursor = cnxn.cursor()

In [ ]:
from fake_useragent import UserAgent
ua = UserAgent()
for i in range(5):
    print(ua.random)
    print('*-----------------*')

### 1.透過商品主類別,次類別爬取所有商品商家之shopid

In [ ]:
cursor.execute("SELECT catid FROM Cat ")
rows = cursor.fetchall()
len(rows)

In [ ]:
shopid = []
for a in range(0, len(rows)):
    b = rows[a]
    c = "".join(b)
    print(a, ':', c)
    categoryids = str(c)

    for p in range(1, 81):  #最多80頁
        limit = str(100)
        newest = str(100 * (p - 1))
        url = 'https://shopee.tw/api/v1/search_items/?by=relevancy&fe_categoryids=' + categoryids + '&limit=' + limit + '&newest=' + newest + '&order=desc&page_type=search&version=1'
        header = {
            'Accept': 'application/json;',
            'User-Agent': 'Googlebot',
            'Connection': 'close'
        }
        r = requests.get(url, headers=header)
        i = r.json()
        shopid = []
        for x in range(0, 100):
            try:
                y = i["items"][x]
                z = y['shopid']
                shopid.append(z)
            except:
                pass
            continue
        rshopid = tuple(set(shopid))  #移除重複
        len(rshopid)
        #存成DataFrame在匯入db
        df = pd.DataFrame(rshopid, columns=['shopid'])
        for row in df.itertuples():
            cursor.execute(
                ' IF NOT EXISTS (SELECT 1 FROM dbo.Shopid2 WHERE shopid = ? ) BEGIN INSERT INTO dbo.Shopid2(shopid)VALUES (?) END',
                row.shopid, row.shopid)

        cnxn.commit()
        print(p, 'done')

### 2.爬取商家資訊,instert db

In [ ]:
cursor.execute(
    "select shopid from Shopid2 where shopid not in (select storeid from StoreCopy)"
)
rows = cursor.fetchall()
len(rows)

In [ ]:
#更改User-Agent
#https://shopee.tw/robots.txt

In [ ]:
for i in range(0, len(rows)):
    try:
        df = []
        a = rows[i]
        b = "".join(a)
        header = {
            'Accept': 'application/json;',
            'User-Agent': 'Googlebot',
            'Connection': 'close'
        }
        url = 'https://shopee.tw/api/v2/shop/get?shopid=' + b + ''
        r = requests.get(url, verify=False)
        i1 = r.json()
        j = i1["data"]
        y = j['account']
        x = j['rating_normal'] + j['rating_bad'] + j['rating_good']
        name = y['username']
        url1 = 'https://shopee.tw/' + name + ''
        if j['is_shopee_verified'] == False and j['is_official_shop'] == True:
            shoptype = '蝦皮商城'
        if j['is_shopee_verified'] == True and j['is_official_shop'] == False:
            shoptype = '蝦皮優選'
        if j['is_shopee_verified'] == False and j['is_official_shop'] == False:
            shoptype = '蝦皮購物'
        timeStamp = j['ctime']
        #now = int(time.time())
        timeArray = time.localtime(timeStamp)
        otherStyleTime = time.strftime("%Y-%m-%d %H:%M:%S", timeArray)
        url2 = 'https://shopee.tw/shop/' + b + '/search'
        r2 = requests.get(url2, headers=header, allow_redirects=False)
        soup = bs(r2.text, 'html.parser')
        pid = soup.find('span', {
            'class': 'shopee-mini-page-controller__total'
        }).text
        lastpage = int(pid) - 1
        url3 = "https://shopee.tw/shop/" + b + "/search?page=" + str(
            lastpage) + "&sortBy=pop"
        r3 = requests.get(url3, headers=header, allow_redirects=False)
        soup = bs(r3.text, 'html.parser')
        c = 0
        for tag in soup.find_all(
                'div', {'class': 'shop-search-result-view__item col-xs-2-4'}):
            c = c + 1
        d = c
        itemp = 30
        ritem = itemp * lastpage + d
        print(i, b, url1, y['username'], j['name'], j['shop_location'],
              shoptype, j['item_count'], j['response_rate'],
              j['cancellation_rate'], j['follower_count'], j['rating_star'], x,
              otherStyleTime, ritem)
        df.append([
            str(b),
            str(url1),
            str(y['username']),
            str(j['name']),
            str(j['description']),
            str(j['shop_location']),
            str(shoptype),
            str(j['item_count']),
            str(j['response_rate']),
            str(j['cancellation_rate']),
            str(j['follower_count']),
            str(j['rating_star']),
            str(j['rating_good']),
            str(j['rating_normal']),
            str(j['rating_bad']),
            str(x),
            str(otherStyleTime),
            str(ritem)
        ])
        df2 = pd.DataFrame(
            df,
            columns=[
                'storeid', 'url', 'username', 'name', 'description',
                'shop_location', 'shoptype', 'item_count', 'response_rate',
                'cancellation_rate', 'follower_count', 'rating_star',
                'rating_good', 'rating_normal', 'rating_bad', 'rating_count',
                'ctime', 'ritem_count'
            ])
        for row in df2.itertuples():
            cursor.execute(
                'INSERT INTO dbo.StoreCopy(storeid,url,username,name,description,shop_location,shoptype,item_count,response_rate,cancellation_rate,follower_count,rating_star,rating_good,rating_normal,rating_bad,rating_count,ctime,ritem_count)VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)',
                row.storeid, row.url, row.username, row.name, row.description,
                row.shop_location, row.shoptype, row.item_count,
                row.response_rate, row.cancellation_rate, row.follower_count,
                row.rating_star, row.rating_good, row.rating_normal,
                row.rating_bad, row.rating_count, row.ctime, row.ritem_count)
            cnxn.commit()
    except:
        pass
    continue

### 3.爬取商家商品資訊,insert db

In [ ]:
#方法一
#資料量越多導致,篩選時間太長
cursor.execute(
    "select storeid from StoreCopy where storeid like '6%' and  storeid not in (select shopid from Shopiditem)"
)
rows = cursor.fetchall()
len(rows)

In [ ]:
#方法二
#透過set篩選
cursor.execute("select shopid from Shopid2 where shopid like '8%' ")
rowA = cursor.fetchall()
y = []
for i in range(0, len(rowA)):
    a = rowA[i]
    b = "".join(a)
    y.append(b)
cursor.execute("select distinct shopid from Item8 ")
rowB = cursor.fetchall()
z = []
for i in range(0, len(rowB)):
    a = rowB[i]
    b = "".join(a)
    z.append(b)
X = set(y) - set(z)
rows = tuple(X)
len(rows)

In [53]:
for i in range(0, len(rows)):
    b = rows[i]
    #b = "".join(a)
    url = 'https://shopee.tw/api/v1/search_items/?by=pop&limit=100&match_id=' + b + '&newest=0&order=desc&page_type=shop&version=2'
    header = {
        'Accept': 'application/json;',
        'User-Agent': 'Googlebot',
        'Connection': 'close'
    }
    r = requests.get(url, headers=header)
    i1 = r.json()
    k = math.ceil(i1['total_count'] / 100)
    print(i1['total_count'], k)

    for m in range(1, k + 1):
        icc = []
        limit1 = str(100)
        newset1 = str(100 * (m - 1))
        url2 = 'https://shopee.tw/api/v1/search_items/?by=pop&limit=' + limit1 + '&match_id=' + b + '&newest=' + newset1 + '&order=desc&page_type=shop&version=2'
        r2 = requests.get(url2, headers=header)
        i2 = r2.json()
        print(url2)
        for n in range(0, 100):  #單一商家所有商品itemid
            try:
                j2 = i2["items"][n]
                itemid = j2['itemid']
                f = str(itemid)
                url3 = 'https://shopee.tw/api/v2/item/get?itemid=' + f + '&shopid=' + b + ''
                r3 = requests.get(url3, headers=header)
                i3 = r3.json()
                ci = i3["item"]['categories'][0]['catid']  #主類別
                ci1 = i3["item"]['categories'][1]['catid']  #次類別
                icc.append([str(b), str(itemid), str(ci), str(ci1)])
            except:
                pass
            continue
        df2 = pd.DataFrame(icc,
                           columns=['shopid', 'itemid', 'catid', 'catid2'])
        for row in df2.itertuples():
            cursor.execute(
                'IF NOT EXISTS (SELECT 1 FROM dbo.Item6 WHERE shopid = ? and itemid = ?) BEGIN INSERT INTO dbo.Item6(shopid,itemid,catid,catid2)VALUES (?,?,?,?) END',
                row.shopid, row.itemid, row.shopid, row.itemid, row.catid,
                row.catid2)
            cnxn.commit()
        print(i, m, str(b), 'done')

135 2
https://shopee.tw/api/v1/search_items/?by=pop&limit=100&match_id=6502065&newest=0&order=desc&page_type=shop&version=2
10 1 6502065 done
https://shopee.tw/api/v1/search_items/?by=pop&limit=100&match_id=6502065&newest=100&order=desc&page_type=shop&version=2
10 2 6502065 done


### 計算商家商品類別數

In [ ]:
print(icc.count(62), icc.count(67), icc.count(100), icc.count(73),
      icc.count(2580), icc.count(64), icc.count(65), icc.count(66),
      icc.count(1657), icc.count(74), icc.count(68), icc.count(63),
      icc.count(70), icc.count(69), icc.count(71), icc.count(1837),
      icc.count(1859), icc.count(1611), icc.count(10076), icc.count(75),
      icc.count(72), icc.count(10000067))
df.append([
    icc.count(62),
    icc.count(67),
    icc.count(100),
    icc.count(73),
    icc.count(2580),
    icc.count(64),
    icc.count(65),
    icc.count(66),
    icc.count(1657),
    icc.count(74),
    icc.count(68),
    icc.count(63),
    icc.count(70),
    icc.count(69),
    icc.count(71),
    icc.count(1837),
    icc.count(1859),
    icc.count(1611),
    icc.count(10076),
    icc.count(75),
    icc.count(72),
    icc.count(10000067)
])